In [33]:
!pip install llmlingua

Defaulting to user installation because normal site-packages is not writeable
  Using cached llmlingua-0.2.2-py3-none-any.whl.metadata (17 kB)
  Using cached accelerate-0.29.3-py3-none-any.whl.metadata (18 kB)
Using cached llmlingua-0.2.2-py3-none-any.whl (30 kB)
Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)


In [14]:
import os

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

In [19]:
import PyPDF2
from sentence_transformers import SentenceTransformer
import numpy as np

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.messages import AIMessage,SystemMessage,HumanMessage

In [16]:
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [18]:
embeddings = AzureOpenAIEmbeddings(model = os.environ["LLM_EMBEDDING_MODEL"],)

In [20]:
# Function to extract text from PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        pdf_reader = PyPDF2.PdfReader(f)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [21]:
vectorstore = FAISS.from_texts(
    [extract_text_from_pdf(r"C:\Users\JANU\OneDrive\Documents\GitHub\ML-AI\Codes\Projects&POC\Citizen_POC\Citizens bank-2023.pdf")], embeddings
)

In [22]:
retriever = vectorstore.as_retriever()

In [23]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [24]:
response = retrieval_chain.invoke("What are the bank’s core products and services?")

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 166139 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [1]:
import os
import PyPDF2
import random
import itertools
import streamlit as st
from io import StringIO
from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI ## LLM model - Chat GPT 3
from langchain_openai import AzureOpenAIEmbeddings##embedding type
from langchain_community.vectorstores import FAISS ## Vector DB
from langchain_community.retrievers import SVMRetriever ### classification model
from langchain.chains import QAGenerationChain ### QA pair Generator 
from langchain.text_splitter import RecursiveCharacterTextSplitter ### for spliting the doc
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks import StdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings
from llmlingua import PromptCompressor
import pandas as pd

In [2]:
#Setting Environment variable
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

In [3]:
def load_docs(files):
    print("`Reading doc ...`")
    all_text = ""
    for file_path in files:
        file_extension = os.path.splitext(file_path.name)[1]
        if file_extension == ".pdf":
            pdf_reader = PyPDF2.PdfReader(file_path)
            text = ""
            # Get the total number of pages in the PDF
            total_pages = len(pdf_reader.pages)
            print(f"Total Pages: {total_pages}")

           
            for page_num in range(total_pages):
                page = pdf_reader.pages[page_num    ]
                text += page.extract_text()
              
            all_text += text
        elif file_extension == ".txt":
            stringio = StringIO(file_path.getvalue().decode("utf-8"))
            text = stringio.read()
            all_text += text
        else:
            print('Please provide txt or pdf.', icon="⚠️")
    return all_text

In [4]:
def create_retriever(_embeddings, splits, retriever_type):
    print("`Retriving ...`")
    if retriever_type == "SIMILARITY SEARCH":
        try:
            vectorstore = FAISS.from_texts(splits, _embeddings)
            retriever = vectorstore.as_retriever(k=5)
        except (IndexError, ValueError) as e:
            print(f"Error creating vectorstore: {e}")
            return
    elif retriever_type == "SUPPORT VECTOR MACHINES":
        retriever = SVMRetriever.from_texts(splits, _embeddings)
    return retriever

In [5]:
def split_texts(text, chunk_size, overlap, split_method):

    # Split texts
    # IN: text, chunk size, overlap, split_method
    # OUT: list of str splits
    
    print("`Splitting doc ...`")  
    split_method = "RecursiveTextSplitter"
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=overlap)

    splits = text_splitter.split_text(text)
    if not splits:
        print("Failed to split document")
        stop()

    return splits

In [6]:
def generate_eval(text, N, chunk):

    # Generate N questions from context of chunk chars
    # IN: text, N questions, chunk size to draw question from in the doc
    # OUT: eval set as JSON list
    
    
    print("`Generating sample questions ...`")
    n = len(text)
    
    starting_indices = [random.randint(0, n-chunk) for _ in range(N)]
    sub_sequences = [text[i:i+chunk] for i in starting_indices]
    ##chain = QAGenerationChain.from_llm(ChatOpenAI(temperature=0))
    chain = QAGenerationChain.from_llm(AzureChatOpenAI(deployment_name=os.getenv("DEPLOYMENT_NAME"),
                                                       openai_api_type='Azure',))
   
    eval_set = []
    for i, b in enumerate(sub_sequences):
        try:
            qa = chain.run(b)
            eval_set.append(qa)
           
        except:
            print('Error generating question %s.' % str(i+1), icon="⚠️")
    eval_set_full = list(itertools.chain.from_iterable(eval_set))
    return eval_set_full

In [16]:
def main():

#split_texts
#create_retriever
#load_docs

    # Load and process the uploaded PDF or TXT files.
    loaded_text = load_docs(uploaded_files)
    print("Documents uploaded and processed.")
    
    # Split the document into chunks
    splits = split_texts(loaded_text, chunk_size=1000,
                             overlap=0, split_method=splitter_type)

    # Display the number of text chunks
    num_chunks = len(splits)
    print(f"Number of text chunks: {num_chunks}")

    ################  Azure OpenAI snippet
    # Create an instance of Azure OpenAI
    print("Interacting with Azure Open AI........................")
    llm = AzureChatOpenAI(
        deployment_name=os.getenv("DEPLOYMENT_NAME"),openai_api_type='Azure',
    )

    retriever = create_retriever(embeddings, splits, retriever_type)


In [10]:
loaded_text

NameError: name 'loaded_text' is not defined